In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz
!tar xf spark-3.3.0-bin-hadoop2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (92.1 kB/s)
Reading package li

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-10-04 02:24:22--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.25MB/s    in 0.7s    

2022-10-04 02:24:24 (1.25 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [34]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://group-project-data-bucket.s3.amazonaws.com/clean_salary_data.csv"
spark.sparkContext.addFile(url)
salary_data_df = spark.read.csv(SparkFiles.get("clean_salary_data.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
salary_data_df.show()

+---+----------+-------------------+----------+-----+--------------------+-----------------------+-----------------+--------------+----------+---------------+-------+---------+-------------+-----+
|_c0|responseid|          timestamp|   company|level|               title|totalyearlycompensation|yearsofexperience|yearsatcompany|basesalary|stockgrantvalue|  bonus|rowNumber|         city|state|
+---+----------+-------------------+----------+-----+--------------------+-----------------------+-----------------+--------------+----------+---------------+-------+---------+-------------+-----+
|  0|        A1|2017-06-07 11:33:00|    ORACLE|   L3|     Product Manager|                 127000|              1.5|           1.5|    107000|        20000.0|10000.0|        1| Redwood City|   CA|
|  1|        A2|2017-06-10 17:11:00|      EBAY| SE 2|   Software Engineer|                 100000|              5.0|           3.0|         0|            0.0|    0.0|        2|San Francisco|   CA|
|  2|        A3

In [35]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://group-project-data-bucket.s3.amazonaws.com/clean_participants_data.csv"
spark.sparkContext.addFile(url)
participants_data_df = spark.read.csv(SparkFiles.get("clean_participants_data.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
participants_data_df.show()

+---+----------+-------------------+-------+------------+--------------+----------------+----------------+----------+------------+----------+----------+----------------+----------+-------------+-------+---------+---------+
|_c0|responseid|          timestamp| gender|otherdetails|Masters_Degree|Bachelors_Degree|Doctorate_Degree|Highschool|Some_College|Race_Asian|Race_White|Race_Two_Or_More|Race_Black|Race_Hispanic|   Race|Education|rowNumber|
+---+----------+-------------------+-------+------------+--------------+----------------+----------------+----------+------------+----------+----------+----------------+----------+-------------+-------+---------+---------+
|  0|        A1|2017-06-07 11:33:00|Unknown|     Unknown|             0|               0|               0|         0|           0|         0|         0|               0|         0|            0|Unknown|  Unknown|        1|
|  1|        A2|2017-06-10 17:11:00|Unknown|     Unknown|             0|               0|               0|  

In [36]:
participants_data_df = participants_data_df.dropna()
participants_data_df.show(200)

+---+----------+-------------------+-------+--------------------+--------------+----------------+----------------+----------+------------+----------+----------+----------------+----------+-------------+-------+---------+---------+
|_c0|responseid|          timestamp| gender|        otherdetails|Masters_Degree|Bachelors_Degree|Doctorate_Degree|Highschool|Some_College|Race_Asian|Race_White|Race_Two_Or_More|Race_Black|Race_Hispanic|   Race|Education|rowNumber|
+---+----------+-------------------+-------+--------------------+--------------+----------------+----------------+----------+------------+----------+----------+----------------+----------+-------------+-------+---------+---------+
|  0|        A1|2017-06-07 11:33:00|Unknown|             Unknown|             0|               0|               0|         0|           0|         0|         0|               0|         0|            0|Unknown|  Unknown|        1|
|  1|        A2|2017-06-10 17:11:00|Unknown|             Unknown|           

In [37]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgressdb.cuwtf0lwxiav.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", 
          "password": "pasgroup3", 
          "driver":"org.postgresql.Driver"}

In [39]:
# Write DataFrame to salary_data table in RDS

salary_data_df.write.jdbc(url=jdbc_url, table='salary_data',mode=mode,properties=config)

In [40]:
# Write DataFrame to salary_data table in RDS 

participants_data_df.write.jdbc(url=jdbc_url, table='participants_data',mode=mode,properties=config)